<a href="https://colab.research.google.com/github/himanshisyadav/Webscraper_NER/blob/main/Markup_LM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 18.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.7 MB/s eta 0:00:00

In [3]:
from transformers import AutoProcessor, AutoModelForTokenClassification, get_scheduler, MarkupLMProcessor
import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import evaluate
import pdb
import glob

In [4]:
!pip install optimum
from optimum.bettertransformer import BetterTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.7/243.7 kB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.7 MB/s eta 0:00:00
  Created wheel for optimum: filename=optimum-1.8.3-py3-none-any.whl size=318069 sha256=b25e4bef72e89e646da69ba732bc52c9ad8c9b62c43d026f280368fe9fef6b5c
  Stored in directory: /root/.cache/pip/wheels/12/a1/9c/a1637559c56182f7a32d41c1a98493fdfd8c765b88874611d8
Successfully built optimum
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3

In [5]:
class MarkupLMDataset(Dataset):
    """Dataset for token classification with MarkupLM."""

    def __init__(self, data, processor=None, max_length=512):
        self.data = data
        self.processor = processor
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # first, get nodes, xpaths and node labels
        item = self.data[idx]
        nodes, xpaths, node_labels = item['nodes'], item['xpaths'], item['node_labels']

        # provide to processor
        encoding = self.processor(nodes=nodes, xpaths=xpaths, node_labels=node_labels, padding="max_length", max_length=self.max_length, return_tensors="pt", truncation=True)

        # remove batch dimension
        encoding = {k: v.squeeze() for k, v in encoding.items()}

        return encoding


In [6]:
def get_labels(label_list, predictions, references):
    # Transform predictions and references tensos to numpy arrays

    y_pred = predictions.detach().clone().numpy()
    y_true = references.detach().clone().numpy()


    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    return  true_predictions, true_labels

In [7]:
def compute_metrics(return_entity_level_metrics=True):
    results = metric.compute()
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

In [8]:
from google.colab import drive
!fusermount -u gdrive
drive.mount("/content/gdrive",force_remount=True)
!ls
%cd gdrive/My Drive/project_folder

fusermount: failed to unmount /content/gdrive: No such file or directory


MessageError: ignored

In [ ]:
!ssh-keygen -t rsa -b 4096

In [ ]:
!ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

In [ ]:
!cat /root/.ssh/id_rsa.pub

In [ ]:
!ssh -T git@github.com

In [ ]:
# ! git clone git@github.com:himanshisyadav/Webscraper_NER.git
! cd /content/gdrive/MyDrive/project_folder/Webscraper_NER/training_data/nodes_xpaths/

In [ ]:
! pwd

In [ ]:
import glob
#Get data from directory ../training_data
files = glob.glob("Webscraper_NER/training_data/nodes_xpaths/Labeled_*.csv")

id2label = {0:'Address', 1:'Beds', 2:'Contact', 3:'NE', 4: 'Name', 5:'Price', 6:'Amenities'}
label2id = {label:id for id, label in id2label.items()}

data = []

for file in files:
    nodes_xpaths = pd.read_csv(file,delimiter=",")
    nodes_xpaths = nodes_xpaths.dropna()
    nodes_xpaths['Labels'].replace(['Address', 'Beds', 'Contact', 'NE', 'Name', 'Price', 'Amenities'], [0, 1, 2, 3, 4, 5, 6], inplace=True)
    nodes_xpaths = nodes_xpaths.rename({"nodes": "nodes", "Labels": "node_labels", "xpaths": "xpaths"}, axis = 'columns')
    cols = nodes_xpaths.columns.tolist()
    cols = [cols[0], cols[2], cols[1]]
    nodes_xpaths = nodes_xpaths[cols]
    data.append({'nodes': nodes_xpaths['nodes'].values.tolist(), 'xpaths': nodes_xpaths['xpaths'].values.tolist(), 'node_labels': nodes_xpaths['node_labels'].values.tolist()})

print(data)

In [ ]:
! set CUDA_LAUNCH_BLOCKING=1

In [ ]:
! pip install sadice

In [ ]:
! pip install seqeval

In [ ]:
from sadice import SelfAdjDiceLoss
processor = MarkupLMProcessor.from_pretrained("microsoft/markuplm-base")
processor.parse_html = False

dataset = MarkupLMDataset(data=data, processor=processor, max_length=1024)

for example in dataset:
    for k,v in example.items():
        print(k,v.shape)

dataloader = DataLoader(dataset, batch_size=1, shuffle=True)


model = AutoModelForTokenClassification.from_pretrained("microsoft/markuplm-base", num_labels=7)

label_list = ["B-" + x for x in list(id2label.values())]
print(label_list)

optimizer = AdamW(model.parameters(), lr=5e-5)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

model.train()

for epoch in range(1):  # loop over the dataset multiple times
    for batch in tqdm(dataloader):
        # get the inputs;
        inputs = {k:v for k,v in batch.items()}

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(**inputs)
        logits = outputs.logits
        # labels = batch["labels"]

        # print(logits.size())
        # print(labels.size())
        # labels = torch.nn.functional.one_hot(labels, num_classes=7)

        # loss = criterion(logits.float().squeeze(), labels.squeeze())
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        print("Loss:", loss.item())

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]
        preds, refs = get_labels(label_list, predictions, labels)
        metric = evaluate.load("seqeval")
        metric.add_batch( predictions=preds, references=refs,)

    eval_metric = compute_metrics()
    print(f"Epoch {epoch}:", eval_metric)
    print(f"Epoch {epoch}:", loss.item)

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
classes = ['Address', 'Beds', 'Contact', 'NE', 'Name', 'Price', 'Amenities']
cf_matrix = multilabel_confusion_matrix(refs, preds)
df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (12,7))
sn.heatmap(df_cm, annot=True)
plt.savefig('output.png')